<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_7/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_ASPP_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация ASPP сети
В этом уроке мы создадим еще еще одну сегментационную FCN модель на основе блока ASPP (Atrous Spatial Pyramid Pooling), который основан на дилатационных свёртках (Atrous Conv, Dilated Conv)

### Загрузка необходимых библиотек
Здесь мы загружаем различне библиотеки, включая TensoFlow.


In [ ]:
import numpy as np

import tensorflow as tf
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.7.0


### Создание ASPP модели
В этом примере мы будем создавать модель `ASPPNet` через собственный класс, наследованный от `tf.keras.Model`. Кроме того, будем использовать возможность модульного создания нейросетей. Сначала создадим отдельно модель для ASPP блока (блок, который содержит только дилатационные свёртки и применяется между энкодером и декодером), которая по сути будет являться самостоятельным слоем. А затем создадим финальную модель `ASPPNet`, в который `ASPPBlock` будет использоваться как слой между Энкодером и Декодером.

В ASPP блоке (`ASPPBlock`) ко входному тензору параллельно применяются обычная свёртка 1x1 и несколько дилатационных свёрток. Затем все эти резульаты конкатенируются. Далее "перемешиваем" каналы полученного тензора с помощью еще одной свёртки 1x1.

В финальной ASPP модели (`ASPPNet`) в качестве Энкодера будем использовать стандартные свёртки и пулинги. Далее в боттлнеке (в середине сети) применим слой ASPPBlock. А в Декодере будем использовать более простую архитектуру: меньше свёрток, повышение размерности с помощью двух билинейных интерполяций (`tf.image.resize`). Кроме того, в сети присутствуют две проброшенные связи.

In [ ]:
class ASPPBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        out = tf.concat([out1, out2, out3, out4], axis=3)
        out = self.conv5(out)
        return out
    
class ASPPNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')

        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

        self.aspp = ASPPBlock()

    def call(self, x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out_enc_mid = out
        out = self.maxpool(out)
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)
        out = self.conv9(out)
        out = self.conv10(out)

        out = self.aspp(out)

        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)

        out_enc_mid = self.conv11(out_enc_mid)

        out = tf.concat([out, out_enc_mid], axis=3)

        out = self.conv12(out)
        out = self.conv13(out)
        out = self.conv14(out)

        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)
        out = tf.nn.sigmoid(out)
        return out
    
model = ASPPNet()

### Задания


**[ЗАДАНИЕ 1]** Вопрос: почему в ASPPNet в первой билинейной интерполяции (`tf.image.resize`) в качестве желаемого выходного размера тензора мы используем размер тензора `out_enc_mid`?


**[ЗАДАНИЕ 2]** Реализуйте пайплайн обучения для модели ASPPNet: подготовка данных, лосс, обучение, тестирование. Используйте материалы из предыдущего практического урока. Обучите модель и сравните время обучения с временем обучения более простых версий FCN из предыдущих уроков.